In [ ]:
import sys
import os
import platform
import datetime as dt

import pandas as pd
import numpy as np
from IPython.display import display, HTML

import hatchet as ht

sys.path.append("../spotdb")
import spotdb

# Load in a single data file and visualize the tree and dataframe

In [ ]:
# Path to a SPOT/Caliper file
input_deploy_dir_str = "../data/cali-demo"
input_run_ids_str = "cDPu64825TuLB5ujG_0.cali" #problem size=10, iter=215, jobsize=1

db = spotdb.connect(input_deploy_dir_str)
runs = input_run_ids_str.split(',')

# Read SPOT/Caliper file into a Hatchet GraphFrame
gfs = ht.GraphFrame.from_spotdb(db, runs)
gf = gfs.pop()

launchdate = dt.datetime.fromtimestamp(int(gf.metadata["launchdate"]))
jobsize = int(gf.metadata.get("jobsize", 1))

print("launchdate: {}, jobsize: {}".format(launchdate, jobsize))

In [ ]:
# Print the tree representation using the default metric
print(gf.tree())

In [ ]:
# Get the man page of different parameters for Hatchet's tree function
help(gf.tree)

In [ ]:
# Print the dataframe of metrics in HTML-format
display(HTML(gf.dataframe.to_html()))

# Alternatively, use standard printing of the dataframe
#print(gf.dataframe)

# Filter a tree

In [ ]:
# Add new column to the dataframe transforming the inclusive time column to a percentage of the max inclusive time
max_time = gf.dataframe["Total time (inc)"].max()
gf.dataframe["pct-of-max"] = gf.dataframe["Total time (inc)"] / max_time

# Print the dataframe of metrics in HTML-format
display(HTML(gf.dataframe.to_html()))

In [ ]:
# Filter the tree to contain only nodes consuming at least 60% of max time
filter_func = lambda x: x["pct-of-max"] > 0.6
filtered_squashed_gf = gf.filter(filter_func,
                                 squash=True)

In [ ]:
# Compare size of input graph and filtered graph
print(f"Input Graph Size                       : {len(gf.graph)}")
print(f"Result (after filter/squash) Graph Size: {len(filtered_squashed_gf.graph)}")

In [ ]:
# Print the resulting tree
print(filtered_squashed_gf.tree(metric_column="pct-of-max"))

# Calulate percent change between nightly test runs

In [ ]:
# Path to SPOT/Caliper files
input_deploy_dir_str = "../data/cali-demo"
# Input file configurations:
#     problemsize=50, iter=800, jobsize=343, 3/24/21 12:36
#     problemsize=50, iter=800, jobsize=343, 3/24/21 12:45
input_run_ids_str = "cQ-CGJlYj-uFT2yv-_1.cali,cQ-CGJlYj-uFT2yv-_2.cali"

db = spotdb.connect(input_deploy_dir_str)
runs = input_run_ids_str.split(',')

# Read SPOT/Caliper files into a Hatchet GraphFrame
gfs = ht.GraphFrame.from_spotdb(db, runs)

gf = gfs[0]   # 3/24/21 12:36
gf2 = gfs[1]  # 3/24/21 12:45

In [ ]:
# Calculate percent change over time
gf3 = (gf2 - gf) / gf

# Use Python to scale the "Total time (inc)" column by 100%
gf3.dataframe["Total time (inc)"] *= 100

In [ ]:
# Use Python to add a new column to the dataframe, computing the absolute value of the "Total time (inc)" column
gf3.dataframe["abs-pct-change"] = abs(gf3.dataframe["Total time (inc)"])

# Print the resulting tree
# Notice that MPI node is red, indicating a high percent change in time
print(gf3.tree(metric_column="abs-pct-change"))

In [ ]:
# Print the resulting dataframe of metrics
# The dataframe now has the new column that was created earlier called "abs-pct-change"
display(HTML(gf3.dataframe.to_html()))

### Quick analysis of percent change over time

We're interested in analyzing how the MPI library changed overtime as compared to the computation part of our application. For this, we will filter our percent change graphframe to look at the communication and the computation part of our code individually.

In [ ]:
# Create a filter for MPI function calls, apply this filter to the graphframe above
filt_mpi_func = lambda x: x["name"].startswith("MPI_")
filter_mpi = gf3.filter(filt_mpi_func, squash=True)

# Print the resulting dataframe of metrics
display(HTML(filter_mpi.dataframe.to_html()))

# Calculate the average percent change across all MPI function calls
print(
    "Avg percent change of MPI functions: {0}%".format(
        round(filter_mpi.dataframe["abs-pct-change"].mean(), 2)
    )
)

In [ ]:
# Create a filter for non-MPI function calls (e.g., computation), apply this filter to the graphframe above
filt_non_mpi_func = lambda x: not x["name"].startswith("MPI_")
filter_non_mpi = gf3.filter(filt_non_mpi_func, squash=True)

# Print the resulting dataframe of metrics
display(HTML(filter_non_mpi.dataframe.to_html()))

# Calculate the average percent change across all non-MPI (e.g., computation) function calls
print(
    "Avg percent change of computation functions: {0}%".format(
        round(filter_non_mpi.dataframe["abs-pct-change"].mean(), 2)
    )
)

# Calculate speedup between two trees

In [ ]:
# Path to SPOT/Caliper files
input_deploy_dir_str = "../data/cali-demo"
# Input file configurations:
#    problem size=10, niter=215, jobsize=1
#    problem size=10, niter=800, jobsize=64
input_run_ids_str = "cDPu64825TuLB5ujG_0.cali,cjDCIuaXAoayBi9Lr_2.cali" 

db = spotdb.connect(input_deploy_dir_str)
runs = input_run_ids_str.split(',')

# Read SPOT/Caliper files into a Hatchet GraphFrame
gfs = ht.GraphFrame.from_spotdb(db, runs)

gf = gfs[0]   # 1 rank
gf2 = gfs[1]  # 64 ranks

In [ ]:
# Print the tree representation using the default metric
print(gf.tree())

In [ ]:
# Print the tree representation using the metric
print(gf2.tree())

In [ ]:
# Compute the speedup from 1 to 64 ranks
gf3 = gf / gf2

In [ ]:
# Print the resulting tree
# Two things to note here:
# 1) The MPI nodes are annotated with a green arrow that points to the right. This indicates that those
#    nodes exist only in the right tree (i.e., 64 ranks). By right tree, we are referring to the position
#    in the equation gf3 = gf / gf2 as shown in the above cell.
# 2) Nodes with good speedup are highlighted in red, but may be preferred to color these nodes in green.
print(gf3.tree())

In [ ]:
# Print resulting tree, but reverse the color scheme, so red identifies nodes with poor scaling (low values)
print(gf3.tree(invert_colormap=True))

In [ ]:
# Print the resulting dataframe of metrics
display(HTML(gf3.dataframe.to_html()))

# Generate a scaling plot

In [ ]:
%matplotlib inline

import re

# problemsize=50, iter=800, jobsize={27, 64, 125, 216, 343}
WEAK_SCALE_CALI_FILES = [
    "cH_Cf7SG68XmGTJie_1.cali",
    "cfN2J75TCzYjMpOcE_1.cali",
    "cSnFyF8eGAduI9X1A_1.cali",
    "cPShLtn_8i6zbKQdD_1.cali",
    "cQ-CGJlYj-uFT2yv-_1.cali",
]

In [ ]:
dataframes = []

input_db_uri_str = "../data/cali-demo"
input_run_ids_str = ",".join(WEAK_SCALE_CALI_FILES)

db = spotdb.connect(input_db_uri_str)
runs = input_run_ids_str.split(',')

# Read SPOT/Caliper files into Hatchet GraphFrame
gfs = ht.GraphFrame.from_spotdb(db, runs)

for idx, gf in enumerate(gfs):
    # Extract the number of ranks from the filename, and add this as a new column in the dataframe
    jobsize = int(gf.metadata.get("jobsize", 1))
    gf.dataframe["nranks"] = jobsize
    
    # Filter the dataframe to match `Calc*` functions that have a duration greater than 15 seconds
    filtered_gf = gf.filter(lambda x: x["Avg time/rank (inc)"] > 15 and x["name"].startswith('Calc'))
  
    # Append the filtered dataframe to a global list of dataframes
    dataframes.append(filtered_gf.dataframe)

# Concatenate list of dataframes into a single dataframe
result = pd.concat(dataframes)  

# Format rank column with leading 0s
result["nranks"] = result["nranks"].apply(lambda x: '{0:0>3}'.format(x))

# Create a line plot of number of ranks vs. inclusive time by function name
pivot_df = result.pivot(index="nranks",
                        columns="name",
                        values="Total time (inc)")
plt = pivot_df.loc[:,:].plot.line(figsize=(10, 7),
                                  legend=True,
                                  fontsize="large")

# Set the plot title and its font size
plt.set_title("Lulesh Weak Scaling of \"Calc*\" Functions\nProblem Size: 50x50x50",
              fontsize="x-large")

# Customize the legend, set the font size of the title and labels
plt.legend(loc='center left',
           bbox_to_anchor=(1, 0.5),
           title="Function Name",
           fontsize="x-large",
           title_fontsize="x-large")

# Set the x-axis label and its font size
plt.set_xlabel("Number of Ranks",
               fontsize="x-large")

# Set the y-axis label and its font size
plt.set_ylabel("Total Inclusive Time (sec)",
               fontsize="x-large")